<a href="https://colab.research.google.com/github/zhus-dika/railways_caclulations/blob/main/railways_calculations_regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐫 Realize regression model

## 🦀 Download data

### 🪰 Install need packages

In [289]:
# ! apt install x13as
! pip install pyexcel pyexcel-xls pyexcel-xlsx

### 🐳 Imports

In [290]:
import pandas as pd
import statsmodels.api as sm
import gdown
from datetime import datetime
import math

from openpyxl import load_workbook

### 🐻‍❄️ Useful methods

In [382]:
def construct_serial_data_by_type(type, df, dct_serial):
  dates = [datetime.strptime(i, '%y-%m') for i in df.columns]
  serial_dates = {i: [] for i in dct_serial[type]}
  for idx, date in enumerate(dates):
    for key, value in serial_dates.items():
      dt = datetime.strptime(key[3:5]+'-'+key[-2:], '%y-%m')
      if date == dt:
        serial_dates[key].append(1)
      else:
        serial_dates[key].append(0)
  return serial_dates

In [292]:
start = 2002
end = 2017

def excel_columns(n):
    columns = []
    for i in range(1, n + 1):
        col = ""
        num = i
        while num > 0:
            num -= 1
            col = chr(65 + (num % 26)) + col
            num //= 26
        columns.append(col)
    return columns

column_names = excel_columns(14 * (end - start + 1))

In [293]:
def fill_transportation_tariffs(table_data, start, end):
  commodities_transportation_tariffs = {datetime.strptime(str(year) + '-' + str(i), '%Y-%m').strftime("%y-%m"): [] for year in range(start, end + 1) for i in range(1, 13)}
  commodities_transportation_tariffs['name'] = []
  name_column = 'A'
  for i in range(1, 82):
    if i % 4 == 0:
      value = table_data[name_column+str(i)].value
      value = value.replace(',', ' ').replace('.', ' ').replace(')', ' ').replace('(', ' ').replace('-', ' ').replace('из ', ' ').replace(' нее:', ' ').replace(' него:', ' ').replace(' них:', ' ').replace("\n", " ").lower()
      name = " ".join(value.split())
      commodities_transportation_tariffs['name'].append(name)

      for column_num in range((end - start + 1) * 12):
        val_volume = table_data[column_names[column_num + 2 + math.floor(column_num / 12)] + str(i + 1)].value
        commodities_transportation_tariffs[datetime.strptime(str(start + math.floor(column_num / 12))  + '-' + str(column_num % 12 + 1), '%Y-%m').strftime("%y-%m")].append(val_volume)
  return commodities_transportation_tariffs

### 🐟 Read data

🐺 Getting indexed adjustment volumes data

In [294]:
url = "https://drive.google.com/uc?id=1nv0pPe2E1NQRzZF-pHHZkTT71CTHc68R"
volumes = 'volumes.xlsx'
gdown.download(url, volumes, quiet=False);


df_volumes = pd.read_excel('volumes.xlsx', index_col='type')

Downloading...
From: https://drive.google.com/uc?id=1nv0pPe2E1NQRzZF-pHHZkTT71CTHc68R
To: /content/volumes.xlsx
100%|██████████| 107k/107k [00:00<00:00, 66.5MB/s]


In [295]:
df_volumes = df_volumes.rename(columns={date: date.strftime('%y-%m') for date in df_volumes.columns})

In [296]:
df_volumes.sample(5)

,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,...,24-03,24-04,24-05,24-06,24-07,24-08,24-09,24-10,24-11,24-12
type,,,,,,,,,,,,,,,,,,,,,
ColOre_adjusted,1.170649,1.088629,1.004364,1.007313,1.017305,1.066902,0.999734,1.022211,0.980279,0.944664,...,1.143482,1.076594,1.104551,0.928161,1.156911,1.121502,1.102615,1.081331,1.025729,1.044812
nFerr,0.972207,1.007611,1.002024,0.984602,1.011958,0.953884,0.984933,0.927702,0.968358,1.012275,...,0.892293,0.986672,1.013474,0.982458,1.017588,0.998086,0.981333,0.987121,0.941258,0.988864
Fert_adjusted,0.913359,0.947042,1.061564,1.099830,1.180900,0.941476,1.071067,1.158551,1.061461,1.114627,...,0.850746,1.033015,1.398331,1.109630,1.147315,1.081237,0.968222,1.776032,1.124368,0.872005
ColOre,1.130754,1.079595,1.006360,0.996961,1.015820,1.081599,0.986914,1.028177,0.970435,0.939595,...,1.129539,1.069297,1.107229,0.980189,1.172391,1.126803,1.096738,1.081936,1.005338,1.026811
Lim_adjusted,1.243567,1.161781,1.088058,1.209383,1.083081,1.137478,0.964124,0.961124,1.191410,1.389540,...,1.177135,1.152923,1.039607,0.994984,0.985720,0.917058,0.757069,0.972525,1.116322,1.017416


🦐 Filter only adjusted rows

In [297]:
adjusted_rows = [i for i in df_volumes.index if len(i.split('_')) > 1]

mask = df_volumes.index.isin(adjusted_rows)
df_volumes = df_volumes[mask]

In [298]:
df_volumes.sample(5)

,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,...,24-03,24-04,24-05,24-06,24-07,24-08,24-09,24-10,24-11,24-12
type,,,,,,,,,,,,,,,,,,,,,
Wheat_adjusted,1.405050,1.335464,1.243198,1.222072,1.201595,1.164400,1.046377,0.921651,0.984873,0.996572,...,0.984923,0.906323,1.012457,1.108902,0.909788,1.013309,1.064729,1.063984,0.962319,1.089922
Cement_adjusted,1.147336,1.102376,0.892257,0.905910,1.308594,1.430011,1.325809,1.229607,1.270885,1.222293,...,0.818200,1.018180,0.966344,0.968121,1.033922,1.064140,1.089983,0.983672,0.948599,1.083497
Oil_adjusted,1.114516,1.124344,1.086534,1.077561,1.160076,1.115107,1.015461,0.994196,1.027111,1.063774,...,0.996553,0.978653,0.955117,0.973056,1.025939,0.997069,1.022066,0.814616,0.927846,0.905966
Iron_adjusted,1.196323,1.137040,1.090749,1.076335,1.193487,1.106854,1.280330,1.251372,1.314330,1.248365,...,1.350003,1.142641,1.207504,1.233599,1.122828,1.392767,1.229503,1.179020,1.171365,0.987085
Black_adjusted,1.107154,1.032615,1.036182,1.121432,0.964139,1.101222,1.080336,1.071287,1.114400,1.020916,...,1.094561,1.033828,1.148415,1.132755,1.118854,1.101313,1.001237,1.074308,1.052041,1.079446


In [299]:
df_volumes.index = [i.split('_')[0] for i in df_volumes.index]

In [300]:
df_volumes.sample(5)

,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,...,24-03,24-04,24-05,24-06,24-07,24-08,24-09,24-10,24-11,24-12
Chem,0.947980,0.922180,0.988208,0.319391,0.741968,0.785920,0.647708,0.915157,1.108482,1.343730,...,1.164998,1.193062,1.096665,1.289898,0.753116,1.005912,1.112207,0.989660,1.401184,1.106079
Coke,1.038652,1.006180,0.993246,0.988889,0.986890,1.026394,0.998445,0.997933,1.003222,0.987501,...,1.208424,1.221924,1.667015,1.185833,1.046100,1.077373,1.053056,1.105203,1.444303,1.190871
Wheat,1.405050,1.335464,1.243198,1.222072,1.201595,1.164400,1.046377,0.921651,0.984873,0.996572,...,0.984923,0.906323,1.012457,1.108902,0.909788,1.013309,1.064729,1.063984,0.962319,1.089922
Drinks,1.058928,1.010184,1.019816,1.107321,1.055743,1.131114,1.126587,1.173688,1.274232,1.168759,...,0.983818,1.005387,1.021985,1.021990,1.050561,1.015551,1.034032,1.077381,1.032528,1.125397
Oil,1.114516,1.124344,1.086534,1.077561,1.160076,1.115107,1.015461,0.994196,1.027111,1.063774,...,0.996553,0.978653,0.955117,0.973056,1.025939,0.997069,1.022066,0.814616,0.927846,0.905966


🐤 Getting indexed adjustment transportation volumes data

In [301]:
url = "https://drive.google.com/uc?id=1jzuze4gNQkryG2up3ruFI19LoASI14X9"
transportation_volumes = 'transportation_volumes.xlsx'
gdown.download(url, transportation_volumes, quiet=False);


df_transportation_volumes = pd.read_excel(transportation_volumes, index_col='type')

Downloading...
From: https://drive.google.com/uc?id=1jzuze4gNQkryG2up3ruFI19LoASI14X9
To: /content/transportation_volumes.xlsx
100%|██████████| 67.8k/67.8k [00:00<00:00, 35.0MB/s]


In [302]:
df_transportation_volumes = df_transportation_volumes.rename(columns={date: date.strftime('%y-%m') for date in df_transportation_volumes.columns})

In [303]:
df_transportation_volumes.head()

,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,...,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12
type,,,,,,,,,,,,,,,,,,,,,
Coal,1.146989,1.271296,2.001057,1.617007,1.497430,2.758001,1.602998,0.851024,0.779384,0.822710,...,1.299334,1.142139,1.127997,1.232185,1.185096,1.113000,0.890330,0.961143,1.088174,0.912056
Iron,1.412507,1.134848,1.154288,1.236893,1.152850,1.289855,1.010562,1.016424,1.126092,1.041267,...,1.027275,1.132663,1.022012,1.012473,1.244028,1.367165,1.146183,1.374477,1.153021,0.979902
Oil,1.001428,0.888155,0.881444,1.181056,1.216150,0.812597,0.862972,0.754016,0.798269,0.892205,...,1.176338,0.807852,0.602213,0.568115,0.555227,0.526765,0.670949,0.827540,0.679410,0.432077
Petrol,1.284108,1.637060,1.612777,1.293651,1.817554,1.618301,1.407716,1.418118,1.159494,1.268705,...,1.318899,0.804209,0.999319,0.876377,1.012419,0.888688,0.895795,1.082105,1.182459,1.007365
Scrap,0.678691,0.532579,1.807040,1.914268,1.586697,0.737438,1.703007,0.809680,1.254796,0.951983,...,1.519347,1.133832,1.079397,0.991456,1.032845,1.302089,1.657707,1.855247,1.647698,0.938190


In [304]:
df_volumes = df_volumes[df_transportation_volumes.columns]
df_volumes.sample(5)

,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,...,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12
Iron,1.196323,1.137040,1.090749,1.076335,1.193487,1.106854,1.280330,1.251372,1.314330,1.248365,...,1.293152,1.152380,0.949399,1.056657,1.106089,1.162248,1.103974,1.048065,1.091935,1.073739
Chem,0.947980,0.922180,0.988208,0.319391,0.741968,0.785920,0.647708,0.915157,1.108482,1.343730,...,2.373460,2.570540,1.727386,1.960698,1.450607,1.147874,0.921086,0.796878,1.059980,1.122388
ColOre,1.170649,1.088629,1.004364,1.007313,1.017305,1.066902,0.999734,1.022211,0.980279,0.944664,...,1.232864,1.192358,1.253334,1.195697,1.105047,1.199743,1.088386,1.055487,1.067826,1.049844
Drinks,1.058928,1.010184,1.019816,1.107321,1.055743,1.131114,1.126587,1.173688,1.274232,1.168759,...,1.028924,0.998467,1.136631,1.114289,1.204199,1.142786,1.060212,1.287790,1.356071,1.213499
Lim,1.243567,1.161781,1.088058,1.209383,1.083081,1.137478,0.964124,0.961124,1.191410,1.389540,...,1.157867,1.386690,0.923051,1.060347,1.172591,1.046441,1.019964,1.004049,0.927571,0.893973


🦢 Filter only adjusted rows

In [305]:
adjusted_rows = [i for i in df_transportation_volumes.index if len(i.split('_')) > 1]

mask = df_transportation_volumes.index.isin(adjusted_rows)
df_transportation_volumes = df_transportation_volumes[mask]

In [306]:
df_transportation_volumes.index = [i.split('_')[0] for i in df_transportation_volumes.index]

In [307]:
df_transportation_volumes.sample(5)

,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,...,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12
Scrap,0.886829,0.585345,1.587357,1.831370,1.557243,0.597326,1.430218,0.868340,1.377285,1.090402,...,1.419160,1.190464,1.119767,0.992376,1.005088,1.119658,1.656623,1.840343,1.737905,0.954867
Black,0.985351,1.047713,1.126439,0.899492,0.972774,0.992390,1.278589,1.169062,1.175883,0.940634,...,0.997840,1.014568,1.065487,0.974720,1.056951,1.021436,1.032156,0.942542,1.247452,1.068753
Oil,1.055691,0.888278,0.905556,1.131349,1.113799,0.782867,0.872343,0.816795,0.865678,0.879591,...,1.256835,0.771495,0.649785,0.581336,0.553317,0.501257,0.579954,0.804585,0.694120,0.487653
ColOre,1.667053,1.111643,1.127678,1.044785,0.964588,0.861963,1.212027,0.905385,0.990382,1.055694,...,1.327676,1.512816,1.199173,1.143847,1.167131,1.109588,1.108317,1.180325,1.202237,1.038911
Drinks,1.361685,1.549208,2.370631,2.039233,2.410323,5.035494,5.775303,1.383743,4.468019,2.898000,...,1.127810,1.120776,0.911704,0.985742,1.036572,0.775076,0.952031,1.051133,1.561849,0.896086


🦧 Prepare SERial data

In [336]:
dct_serial = {
    'Coal': ["SER0306", "SER0406", "SER1006", "SER1311", "SER1312", "SER1412", "SER1501", "SER1510", "SER1706"],
    'Iron': ["SER0505", "SER0604", "SER0605", "SER0606", "SER0607", "SER0608", "SER0705", "SER0811", "SER0910", "SER0911", "SER0912", "SER1001", "SER1004", "SER1312", "SER1401", "SER1501", "SER1507", "SER1509", "SER1705", "SER1706", "SER1707"],
    'Oil': ["SER0402", "SER0409", "SER0601", "SER0602", "SER0603", "SER0608", "SER0610", "SER1207", "SER1208", "SER1311", "SER1404", "SER1607", "SER1703", "SER1708"],
    'Petrol': ["SER0405", "SER0510", "SER0706", "SER0912", "SER1009", "SER1212", "SER1401", "SER1501", "SER1505", "SER1603", "SER1604"],
    'Constr': ["SER0501", "SER0803", "SER0811", "SER1002", "SER1011", "SER1103", "SER1301", "SER1401", "SER1502", "SER1602", "SER170"],
    'Chem': ["SER0307", "SER0411", "SER0807", "SER0901", "SER0903", "SER0904", "SER0905", "SER0909", "SER0910", "SER0911", "SER1001", "SER1009", "SER1010", "SER1104", "SER1302", "SER1303", "SER1401", "SER1501", "SER1611"],
    'Black': ["SER0603", "SER1101", "SER1501", "SER1701"],
    'Cement': ["SER0411", "SER0501", "SER0808", "SER1001", "SER1004", "SER1006", "SER1007", "SER1008", "SER1011", "SER1312", "SER1501", "SER1701"],
    'Coke': ["SER0501", "SER0701", "SER0710", "SER0901", "SER1003", "SER1004", "SER1012", "SER1102", "SER1201", "SER1203", "SER1204", "SER1409", "SER1412", "SER1501", "SER1703", "SER1707"],
    'ColOre': ["SER0407", "SER0501", "SER0901", "SER1001", "SER1206", "SER1302", "SER1312", "SER1401", "SER1402", "SER1501", "SER1502", "SER1505", "SER1606"],
    'Corn': ["SER0603", "SER0604", "SER0611", "SER0708", "SER0906", "SER0908", "SER0909", "SER0910", "SER1104", "SER1201", "SER1203", "SER1208", "SER1209", "SER1210", "SER1501", "SER1507"],
    'Drinks': ["SER0401", "SER0508", "SER0701", "SER0801", "SER1009", "SER1104", "SER1306", "SER1312", "SER1401", "SER1501", "SER1509"],
    'Earth': ["SER0401", "SER0503", "SER0704", "SER0710", "SER0812", "SER0903", "SER1002", "SER1101", "SER1301", "SER1401", "SER1403", "SER1501", "SER1503", "SER1505", "SER1701"],
    'Fert': ["SER0409", "SER0510", "SER0511", "SER0702", "SER0810", "SER1010", "SER1011", "SER1107", "SER1201", "SER1710"],
    'Lim': ["SER0702", "SER1002", "SER1003", "SER1010", "SER1011", "SER1102", "SER1312", "SER1401", "SER1501", "SER1701", "SER1711"],
    'nFerr': ["SER0405", "SER0406", "SER0511", "SER0602", "SER0603", "SER0704", "SER0709", "SER0806", "SER0810", "SER0906", "SER0912", "SER1001", "SER1004", "SER1005", "SER1207", "SER1401", "SER1405", "SER1412", "SER1501", "SER1504", "SER1601", "SER1709", "SER1710", "SER1712"],
    'Other': ["SER0912", "SER1007", "SER1012", "SER1312", "SER1401", "SER1501"],
    'Scrap': ["SER0401", "SER0404", "SER0405", "SER0406", "SER0407", "SER0411", "SER0603", "SER0604", "SER0810", "SER0910", "SER0911", "SER0912", "SER1101", "SER1111", "SER1502", "SER1606", "SER1608", "SER1701", "SER1702"],
    'Slag': ["SER0402", "SER0704", "SER0705", "SER0707", "SER0708", "SER0911", "SER1001", "SER1101", "SER1102", "SER1109", "SER1110", "SER1511", "SER1601", "SER1612"],
    'Wheat': ["SER0806", "SER0912", "SER1201", "SER1403"]
}

🦑 Prepare indexed tariff data

In [309]:
url = "https://drive.google.com/uc?id=1cPzbXML9AOPs4TeNqj-jDf_vEVgugG5q"
tariff = 'tariff.xlsx'
gdown.download(url, tariff, quiet=False);

Downloading...
From: https://drive.google.com/uc?id=1cPzbXML9AOPs4TeNqj-jDf_vEVgugG5q
To: /content/tariff.xlsx
100%|██████████| 530k/530k [00:00<00:00, 37.3MB/s]


In [310]:
wb = load_workbook(tariff)

main_sheet = 'Тариф'
commodities_transportation_tariffs = {}

In [311]:
column_names = excel_columns(14 * (end - start + 1))

commodities_transportation_tariffs =  fill_transportation_tariffs(wb[main_sheet], start, end)

In [312]:
df_transportation_tariff = pd.DataFrame(commodities_transportation_tariffs)

In [313]:
df_transportation_tariff.head()

,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,...,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,уголь
1,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,железная руда
2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,нефть сырая
3,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,нефтепродукты
4,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,строительные грузы


In [314]:
dct_names = {'Coal': 'Уголь',
            'Iron': 'Железная руда',
            'Earth': 'Земля, песок, глина и камни природные',
            'Lim': 'Известняк для флюсования',
            'Coke': 'Кокс каменноугольный',
            'Black': 'Чер. металлы',
            'Scrap': 'Лом черных металлов',
            'Wheat': 'Мука пшеничная',
            'Oil': 'Нефть сырая',
            'Petrol': 'Нефтепродукты',
            'Drinks': 'Соки и напитки',
            'Fert': 'Хим. и мин. удобрения',
            'nFerr': 'Цвет. металлы',
            'ColOre': 'Цветная руда',
            'Chem': 'Химикаты и сода',
            'Cement': 'Цемент'
            }

In [315]:
for key in dct_names.keys():
  dct_names[key] = " ".join(dct_names[key].replace(',', ' ').replace('.', ' ').replace(')', ' ').replace('(', ' ').replace('-', ' ').lower().split())

In [316]:
common_names = set(dct_names.keys()) & set(df_volumes.index)

### Select data with common names

In [317]:
df_volumes = df_volumes[df_volumes.index.isin(common_names)]

df_transportation_volumes = df_transportation_volumes[df_transportation_volumes.index.isin(common_names)]

In [318]:
df_transportation_tariff['type'] = ''

for idx, row in df_transportation_tariff.iterrows():
  for key in dct_names.keys():
    if dct_names[key] == row['name']:
      df_transportation_tariff.loc[idx, 'type'] = key

In [319]:
df_transportation_tariff.sample(5)

,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,...,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,name,type
16,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,мука пшеничная,Wheat
6,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,лом черных металлов,Scrap
14,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,известняк для флюсования,Lim
11,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,чер металлы,Black
19,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,соки и напитки,Drinks


In [320]:
df_transportation_tariff = df_transportation_tariff[df_transportation_tariff['type'].isin(common_names)]

In [321]:
df_transportation_tariff.sample(5)

,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,...,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,name,type
17,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,кокс каменноугольный,Coke
6,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,лом черных металлов,Scrap
16,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,мука пшеничная,Wheat
9,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,цветная руда,ColOre
7,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,химикаты и сода,Chem


In [322]:
df_transportation_tariff.set_index('type', inplace=True)

In [323]:
df_transportation_tariff.sample(5)

,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,...,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,name
type,,,,,,,,,,,,,,,,,,,,,
Wheat,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,мука пшеничная
Petrol,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,нефтепродукты
Chem,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,химикаты и сода
Lim,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,известняк для флюсования
Scrap,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,лом черных металлов


🐰 Convert all data 2 float

In [324]:
for column_num in df_transportation_tariff.columns:
  if column_num != 'name':
      df_transportation_tariff[column_num] = df_transportation_tariff[column_num].astype(float)
      df_transportation_tariff[column_num] = df_transportation_tariff[column_num] / 100 + 1.0

In [325]:
df_transportation_tariff.sample(5)

,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,...,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,name
type,,,,,,,,,,,,,,,,,,,,,
Oil,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,нефть сырая
Chem,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,химикаты и сода
Lim,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,известняк для флюсования
Petrol,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,нефтепродукты
nFerr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,цвет металлы


🦓 Filter years from 2003

In [351]:
df_transportation_tariff

,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,...,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,name
type,,,,,,,,,,,,,,,,,,,,,
Coal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,уголь
Iron,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,железная руда
Oil,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,нефть сырая
Petrol,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,нефтепродукты
Scrap,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,лом черных металлов
Chem,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,химикаты и сода
Fert,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,хим и мин удобрения
ColOre,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,цветная руда
nFerr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.02,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,цвет металлы


In [353]:
df_transportation_tariff = df_transportation_tariff[df_transportation_volumes.columns]

## 🦚 Run regression model

### 🦤 Prepare data to model

In [419]:
commodity_type = 'Wheat'

type_dct = {
    "Y": df_transportation_volumes.loc[commodity_type],   # Transport Volume Growth Index
    "T": df_transportation_tariff.loc[commodity_type],   # Tariff growth index
    "X": df_volumes.loc[commodity_type]   # Production growth index
}
# dummy variables
dct_serial_data = construct_serial_data_by_type(commodity_type, df_volumes, dct_serial)
for key, value in dct_serial_data.items():
  type_dct[key] = value
df_type = pd.DataFrame(type_dct)

In [420]:
df_type.sample(5)

,Y,T,X,SER0806,SER0912,SER1201,SER1403
05-05,2.515624,1.0,1.419850,0,0,0,0
14-09,0.998849,1.0,0.938658,0,0,0,0
17-07,1.138072,1.0,1.019465,0,0,0,0
16-03,1.632323,1.0,1.072286,0,0,0,0
05-06,2.633511,1.0,1.565333,0,0,0,0


🐂 Add lag

In [421]:
df_type["Y_lag"] = df_type["Y"].shift(1)

In [422]:
df_type.sample(5)

,Y,T,X,SER0806,SER0912,SER1201,SER1403,Y_lag
05-04,2.354364,1.0,1.310970,0,0,0,0,2.191812
08-05,1.867812,1.0,1.057137,0,0,0,0,0.976424
10-10,0.877221,1.0,1.030375,0,0,0,0,1.448754
06-12,1.323279,1.0,1.049426,0,0,0,0,1.322605
07-01,1.723448,1.0,1.250141,0,0,0,0,1.323279


In [423]:
df_type = df_type.dropna()

In [424]:
Y = df_type["Y"]
X = df_type[[i for i in df_type.columns if i != 'Y']]

# Добавляем константу для регрессии
X = sm.add_constant(X)

# Строим регрессию
model = sm.OLS(Y, X).fit()

# Выводим результаты
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.613
Method:                 Least Squares   F-statistic:                     41.33
Date:                Wed, 12 Feb 2025   Prob (F-statistic):           1.15e-33
Time:                        13:08:48   Log-Likelihood:                -70.417
No. Observations:                 179   AIC:                             156.8
Df Residuals:                     171   BIC:                             182.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1058      0.916      0.115      0.9